In [202]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px

In [2]:
def get_historical_data(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m',
                      timezone='auto',
                      model='best_match',
                      start_date='1991-01-01',
                      end_date='2020-12-31'):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "start_date": start_date,
        "end_date": end_date
    }


    resp = r.get("https://archive-api.open-meteo.com/v1/archive",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time'], format='%Y-%m-%dT%H:%M')

    data = data.dropna()

    return data


def compute_climatology(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m',
                      timezone='auto',
                      model='best_match',
                      start_date='1991-01-01',
                      end_date='2020-12-31',
                      year=2020):
    
    data = get_historical_data(latitude, longitude, variables,
                               timezone, model, start_date, end_date)
    data.resample('3H', on='time').first()
    data['doy'] = data.time.dt.strftime("%m%d")
    mean = data.groupby([data.doy, data.time.dt.hour]).mean(numeric_only=True).rename_axis(['doy','hour']).reset_index()
    # Create a fake date just in case
    mean['dummy_date'] = pd.to_datetime(mean['doy'] + str(year) + "T" + mean['hour'].astype(str).str.zfill(2) + "00", format='%m%d%YT%H%M')
    
    return mean

def get_ensemble_data(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m,cloudcover,rain,snowfall,precipitation',
                      timezone='auto',
                      model='icon_seamless',
                      from_now=True):
    """
    Get the ensemble data
    """
    if model == 'icon_seamless':
        forecast_days = 7
    else:
        forecast_days = 14
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://ensemble-api.open-meteo.com/v1/ensemble",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'])

    data = data.dropna()
    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime('now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

In [3]:
ens_forecast = get_ensemble_data(variables='temperature_2m')
clima = compute_climatology(variables='temperature_2m')

In [ ]:
var = 'temperature_2m'

traces = []
for col in ens_forecast.columns[ens_forecast.columns.str.contains(var)]:
    # First do the first 48 hrs
    traces.append(
        go.Scatter(
            x=ens_forecast.loc[ens_forecast.time <= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'),'time'],
            y=ens_forecast.loc[ens_forecast.time <= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), col],
            mode='lines+markers',
            name=col,
            marker=dict(size=4),
            line=dict(width=1),
            showlegend=False),
    )
# Then the remaining as shade

traces.append(go.Scatter(
    x=ens_forecast.loc[ens_forecast.time >= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), 'time'],
    y=ens_forecast.loc[ens_forecast.time >= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), ens_forecast.columns.str.contains(var)].min(axis=1),
    mode='lines',
    line=dict(color='grey'),
    name='Minimum',
    showlegend=False
))

# Add the upper bound line trace
traces.append(go.Scatter(
    x=ens_forecast.loc[ens_forecast.time >= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), 'time'],
    y=ens_forecast.loc[ens_forecast.time >= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), ens_forecast.columns.str.contains(var)].max(axis=1),
    mode='lines',
    line=dict(color='grey'),
    fill='tonexty',  # Fill area between traces
    name='Maximum',
    showlegend=False
))

# Now add climatology 

# ens_forecast['doy'] = ens_forecast['time'].dt.strftime("%m%d")
# ens_forecast['hour'] = ens_forecast['time'].dt.hour
# clima = clima.merge(ens_forecast[['doy','hour','time']], left_on=['doy','hour'], right_on=['doy','hour'])

traces.append(
        go.Scatter(
            x=clima['time'],
            y=clima[var],
            mode='lines',
            name='climatology',
            line=dict(width=4, color='rgba(0, 0, 0, 0.2)', dash='dot'),
            showlegend=False)
    )


fig = go.Figure(traces)

fig.update_xaxes(minor=dict(ticks="inside", showgrid=True))

fig.update_layout(
            xaxis=dict(showgrid=True,
                       range=[ens_forecast['time'].min(),ens_forecast['time'].max()]),
            yaxis=dict(showgrid=True),
            margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
            template='plotly_white',
        )

fig.update_xaxes(showgrid=True, gridwidth=4)
fig.update_yaxes(showgrid=True, gridwidth=4)

fig.update_xaxes(minor=dict(ticks="inside", showgrid=True, gridwidth=3))

fig.show()

In [225]:
fig = px.imshow(
    ens_forecast.loc[:, ens_forecast.columns.str.contains(var)].T,
    x=ens_forecast['time'],
                text_auto=True,
                color_continuous_scale='RdBu_r',
                origin='lower')

fig.update_yaxes(visible=False, showticklabels=False)

fig.update_layout(
            xaxis=dict(showgrid=True,
                       range=[ens_forecast['time'].min(),ens_forecast['time'].max()]),
            yaxis=dict(showgrid=True),
            margin={"r": 3, "t": 3, "l": 3, "b": 3},
            template='plotly_white',
        )
fig.update_coloraxes(showscale=False)
fig.show()